In [ ]:
import random
import pandas as pd

def simulateRecall(iterations: int):
    ground_truth: set[int] = set(range(iterations))
    dynamic_callgraph: set[int] = set()
    static_callgraph: set[int] = set(random.sample(list(range(int(iterations*1.5))), int(iterations/2)))
    bounds = []
    for idx in range(iterations):
        if(random.choice([True,True,True,True,True,True,True,False])):
            dynamic_callgraph.add(idx)
        bounds.append(compute_bounds(ground_truth,dynamic_callgraph, static_callgraph))
    return pd.DataFrame.from_records(bounds)

def compute_bounds(ground_truth: set[int], dynamic_callgraph: set[int], static_callgraph: set[int]):
    if dynamic_callgraph:
        recall_sd = float(len(static_callgraph.intersection(dynamic_callgraph))) / float(len(dynamic_callgraph))
    else:
        recall_sd = float(0)
    recall_sg = float(len(static_callgraph.intersection(ground_truth))) / float(len(ground_truth))
    recall_dg = float(len(dynamic_callgraph.intersection(ground_truth))) / float(len(ground_truth))
    return {"lower_bound": recall_sd * recall_dg, 
            "recall_sg": recall_sg,
            "recall_sd": recall_sd,
            "upper_bound": 1 - (1 - recall_sd) * recall_dg}

In [ ]:
simulateRecall(10)

In [ ]:
import seaborn as sns

data = simulateRecall(250)
data['id'] = data.index
data = data.melt(
    id_vars = ['id'],
    value_vars=['lower_bound', 'recall_sg', 'recall_sd', 'upper_bound'], 
    var_name='bound',
    value_name='value')
sns.lineplot(data=data, x='id', y='value', hue='bound')